CÓDIGO PRINCIPAL


Autores:
- Belda Martínez, Marcos
- Espert Cornejo, Ángela
- Francés Llimerá, Lourdes

CALIBRACIÓN DE CÁMARAS Y SISTEMA ESTÉREO
- Si los archivos de calibración no existen, se inicia automáticamente el proceso de calibración
- Para cada cámara:
    - Se detectan las esquinas en imágenes de un tablero de ajedrez
    - Se calcula la matriz intrínseca (K) y los coeficientes de distorsión (dist)
- Se calibra el sistema estéreo:
    - Se calcula la relación entre las dos cámaras (rotación R, traslación T y matrices E y F)
    - Se guardan los resultados en archivos ".npz" para su posterior uso

EJECUCIÓN DEL SISTEMA

1) Se verifica si existen archivos de calibración
    - Si faltan, se realiza la calibración de las cámaras y sistema estéreo

2) Se inicia la interfaz de triangulación
    - Selección de puntos y cálculo de coordenadas 3D

3) Se realiza la consulta por terminal
    - "Indica con la mano cuántos ítems desea recoger. Si desea salir, pulse ESC"

4) Detección de fingers
    - Número de fingers levantados X

5) Se confirma la selección por terminal
    - "Si desea X ítems, pulse ENTER. En caso contrario, pulse ESC"
        - Si se pulsa ENTER, se pasa al paso 6
        - Si se pulsa ESC, se finaliza el programa

6) Se escanea el escenario para obtener el centroide del pentágono verde
    - Opción 1: captura de camara única (no maneja cambios de posicionamiento)
    - Opción 2: tracking continuo hasta finalizar el proceso 
    - Opción 3: captura de camara cada vez que se va a recoger un ítem (tras cada pick & place)

7) Comunicación con RoboDK del centroide del pentágono mediante MQTT

8) Simulación del movimiento de pick & place X veces
    - Se hace hide y show del ítem para simular su recogida y la aparición del siguiente ítem

In [1]:
# System libraries and file processing
import os                                   # Handling of paths and files in the operating system
from glob import glob                       # Searching for files based on patterns (e.g., *.jpg)

# Computer vision and matrix libraries
import cv2                                  # OpenCV: image and video processing
import numpy as np                          # Numerical operations and array handling

# Libraries for additional tasks
import threading                            # For multithreading programming
import queue                                # For handling data queues (e.g., for multithreading)
import pybullet as p                        # Physics simulation
import time                                 # Time handling and delays

# MediaPipe for computer vision tasks (body tracking, etc.)
# from mediapipe.tasks.python import vision   # Classes and functions for vision-related tasks
# from mediapipe.tasks import python          # Additional Python-specific utilities (redundant in this context)
import mediapipe as mp                      # Core MediaPipe module for building and running ML pipelines

# Libraries for MQTT communication
import paho.mqtt.client as mqtt             # MQTT client for network communication
import json                                 # Handling data in JSON format

In [2]:
# Base path of the project
BASE_PATH = r"C:\Users\137ma\Desktop\Trabajo_Final"

# Path to the "data" folder within the project
DATA_PATH = os.path.join(BASE_PATH, "data")

# Create the "data" folder if it does not exist
if not os.path.exists(DATA_PATH):
    os.makedirs(DATA_PATH)
    print(f"folder created: {DATA_PATH}")

# Paths to the calibration files within the "data" folder
CALIBRATION_PATH_L = os.path.join(DATA_PATH, "calibration_camera_left.npz")
CALIBRATION_PATH_R = os.path.join(DATA_PATH, "calibration_camera_right.npz")
CALIBRATION_PATH_STEREO = os.path.join(DATA_PATH, "calibration_stereo.npz")

# Directories where the images for calibrating each camera are stored
CALIBRATION_IMG_PATH_L = os.path.join(BASE_PATH, "left_calibration_images")
CALIBRATION_IMG_PATH_R = os.path.join(BASE_PATH, "right_calibration_images")
STEREO_IMG_PATH_L = os.path.join(BASE_PATH, "stereo_images_L")
STEREO_IMG_PATH_R = os.path.join(BASE_PATH, "stereo_images_R")

# Size of the chessboard pattern used for calibration (number of inner corners)
BOARD_SIZE = (9, 6)

# Physical size of each square in the pattern (in cm, mm, etc.)
SQUARE_SIZE = 2.6

# Valid image extensions for reading
EXTENSION = ["jpg", "jpeg", "bmp"]

# URL for the cameras
CAM_URL_L = "http://192.168.1.132:8080"    # URL for the left camera
CAM_URL_R = "http://192.168.1.129:8080"    # URL for the right camera

# Size of the camera feed
WIDTH = 640     # Width of the camera feed
HEIGHT = 480    # Height of the camera feed

In [3]:
class cameraCalibration:
    """
    A class for calibrating cameras individually and in stereo pairs.
    """
    def __init__(self):
        """
        Initializes the camera calibration class with default parameters.
        """
        self.board_size = BOARD_SIZE        # Size of the chessboard pattern (number of inner corners)
        self.square_size = SQUARE_SIZE      # Physical size of each square in the pattern
        self.img_ext = EXTENSION            # Valid image extensions for reading

    def individual_calibration(self, name, folder, save_path):
        """
        Calibrates a single camera using a set of images of a chessboard pattern.
        """
        # Display start message
        print("\n\n")
        print(f"Calibration file for camera {name} not found.")
        print(f"Starting calibration for camera {name}...")

        # Define real-world 3D points of the chessboard (same for all images)
        objp = np.zeros((self.board_size[0] * self.board_size[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:self.board_size[0], 0:self.board_size[1]].T.reshape(-1, 2)
        objp *= self.square_size  # Scale to the actual physical size

        objpoints = []  # List of real-world 3D points
        imgpoints = []  # List of detected 2D points in images

        # Load all images with valid extensions
        img = []
        for ext in self.img_ext:
            img.extend(sorted(glob(os.path.join(folder, f"*.{ext}"))))

        # Check if images were found
        if not img:
            print(f"ERROR: No images found in: {folder}")
            return False

        print(f"Found {len(img)} images for calibrating camera {name}.")

        # Iterate over each image and detect chessboard corners
        for idx, path in enumerate(img):
            img = cv2.imread(path)
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            ret, corners = cv2.findChessboardCorners(gray, self.board_size, None)

            if ret:
                # If corners are detected, refine them and save
                criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
                corners = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
                objpoints.append(objp)
                imgpoints.append(corners)
            else:
                print(f"ERROR: Corners not detected in image: {os.path.basename(path)}")

        # At least 5 valid images are needed for calibration
        if len(objpoints) < 5:
            print(f"Only detected patterns in {len(objpoints)} images. At least 5 are required.")
            return False

        print(f"Using {len(objpoints)} valid images for calibration...")

        # Perform calibration using the detected points
        img_shape = gray.shape[::-1]
        ret, K, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, img_shape, None, None)

        # Display the projection matrix for the first image
        R = cv2.Rodrigues(rvecs[0])[0]
        RT = np.concatenate((R, tvecs[0]), axis=1)
        P = K @ RT
        print(f"\nProjection matrix for camera {name}:")
        print(np.array2string(P, formatter={'float_kind': lambda x: f"{x:8.3f}"}))

        # Calculate and display the reprojection error for each image
        total_error = 0
        for i in range(len(objpoints)):
            projected_points, _ = cv2.projectPoints(objpoints[i], rvecs[i], tvecs[i], K, dist)
            error = cv2.norm(imgpoints[i], projected_points, cv2.NORM_L2) / len(projected_points)
            total_error += error
            print(f"Reprojection error for img {i+1}: {error:.4f} px")

        # Calculate the average reprojection error
        reproj_error_avg = total_error / len(objpoints)
        print(f"\nAverage reprojection error for camera {name}: {reproj_error_avg:.4f} px")

        # Save calibration parameters to a file
        np.savez(save_path, K=K, dist=dist)
        print(f"Parameters saved to: {save_path}")
        print("\n\n")
        return True

    def stereo_calibration(self, path_R, path_L, dir_R, dir_L, save_path):
        """
        Calibrates a stereo camera system using individual calibration parameters.
        """
        print("\n\n")
        print("Calibrating stereo system...")

        # Load individual calibration parameters
        try:
            right = np.load(path_R)
            left = np.load(path_L)
            K1, D1 = right["K"], right["dist"]
            K2, D2 = left["K"], left["dist"]
        except:
            print("ERROR: Could not load individual calibrations.")
            return False

        # Load pairs of images from both cameras
        images_r = []
        images_l = []
        for ext in self.img_ext:
            images_r.extend(sorted(glob(os.path.join(dir_R, f"*.{ext}"))))
            images_l.extend(sorted(glob(os.path.join(dir_L, f"*.{ext}"))))

        # Check if there are enough valid pairs
        if len(images_r) != len(images_l) or len(images_r) < 5:
            print("The stereo image folders do not contain enough valid pairs or are unbalanced.")
            return False

        # Generate 3D points of the chessboard pattern
        objp = np.zeros((self.board_size[0]*self.board_size[1], 3), np.float32)
        objp[:, :2] = np.mgrid[0:self.board_size[0], 0:self.board_size[1]].T.reshape(-1, 2)
        objp *= self.square_size

        objpoints, imgpoints_l, imgpoints_r = [], [], []

        # Detect corners in each pair of images (left and right)
        for img1, img2 in zip(images_l, images_r):
            im_l = cv2.imread(img1)
            im_r = cv2.imread(img2)
            gray_l = cv2.cvtColor(im_l, cv2.COLOR_BGR2GRAY)
            gray_r = cv2.cvtColor(im_r, cv2.COLOR_BGR2GRAY)

            ret1, corners_l = cv2.findChessboardCorners(gray_l, self.board_size, None)
            ret2, corners_r = cv2.findChessboardCorners(gray_r, self.board_size, None)

            if ret1 and ret2:
                objpoints.append(objp)
                imgpoints_l.append(corners_l)
                imgpoints_r.append(corners_r)

        # Check if there are enough valid pairs
        if len(objpoints) < 5:
            print("Not enough valid pairs for stereo calibration.")
            return False

        # Perform stereo calibration with fixed individual parameters
        img_shape = gray_l.shape[::-1]
        criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 100, 1e-5)
        flags = cv2.CALIB_FIX_INTRINSIC  # Do not recalculate K1 and K2

        # Calibrate stereo: obtain rotation matrix, translation vector, essential and fundamental matrices
        _, _, _, _, _, R, T, E, F = cv2.stereoCalibrate(
            objpoints, imgpoints_l, imgpoints_r,
            K1, D1, K2, D2, img_shape,
            criteria=criteria, flags=flags
        )

        # Display the results
        print(f"\nRotation matrix R:\n{R}")
        print(f"\nTranslation vector T:\n{T}")
        print(f"\nEssential matrix E:\n{E}")
        print(f"\nFundamental matrix F:\n{F}")

        # Save the results to a file
        np.savez(save_path, K1=K1, D1=D1, K2=K2, D2=D2, R=R, T=T)
        print(f"\nStereo calibration saved to: {save_path}")
        print("\n\n")
        return True

In [ ]:
class stereoInterface:
    """
    A class for handling stereo camera operations, gesture detection, and robot communication.
    """
    
    def __init__(self):
        """
        Initializes the stereo interface with camera URLs and MQTT settings.
        """
        print("\n\n")
        print("Initializing video")
        print("\n\n")
        
        self.window = "Main Program"
        self.height = 480
        self.width = 640

        self.cap_L = cv2.VideoCapture(CAM_URL_L)
        self.cap_R = cv2.VideoCapture(CAM_URL_R)

        self.frame_right = None
        self.frame_left = None
        
        self.stopThreads = False            # Flag to stop threads
        self.lock_frame = threading.Lock()  # Lock for thread-safe frame access
        
        self.ip = "broker.emqx.io"               # IP address for MQTT communication
        self.sendTopic = "robodk/position"  # MQTT topic for sending positions
        self.receiveTopic = "robodk/state"  # MQTT topic for receiving robot state
        self.received = False               # Flag to check if a message has been received
        self.robotState = True              # State of the robot

        self.fingers = -1
        self.position = None
        self.gestureOn = True
        self.triangulationOn = True
        
    def loadCalibration(self):
        """
        Loads stereo calibration parameters from a file.
        """
        try:
            data = np.load(CALIBRATION_PATH_STEREO)
            self.K1, self.D1 = data["K1"], data["D1"]   # Left camera intrinsic matrix and distortion coefficients
            self.K2, self.D2 = data["K2"], data["D2"]   # Right camera intrinsic matrix and distortion coefficients
            self.R, self.T = data["R"], data["T"]       # Rotation and translation matrices
        except:
            raise RuntimeError("ERROR: Could not load stereo calibration.")

    def capture_L(self):
        """
        Captures frames from the left camera.
        """
        while not self.stopThreads and self.cap_L.isOpened():
            ret, frame = self.cap_L.read()
            if ret:
                with self.lock_frame:
                    self.frame_left = frame.copy()     # Store the captured frame
                     
    def capture_R(self):
        """
        Captures frames from the right camera.
        """
        while not self.stopThreads and self.cap_R.isOpened():
            ret, frame = self.cap_R.read()
            if ret:
                with self.lock_frame:
                    self.frame_right = frame.copy()    # Store the captured frame
      
    def fingerCount(self, hand_landmarks):
        """
        Counts the number of fingers raised in a hand.
        """
        finger_tips_ids = [4, 8, 12, 16, 20]    # IDs of finger tips
        count = 0
        # Thumb
        if hand_landmarks.landmark[finger_tips_ids[0]].x < hand_landmarks.landmark[finger_tips_ids[0] - 1].x:
            count += 1
        # Other fingers
        for tip_id in finger_tips_ids[1:]:
            if hand_landmarks.landmark[tip_id].y < hand_landmarks.landmark[tip_id - 2].y:
                count += 1
        return count
    
    def gestureDetection(self):
        """
        Detects hand gestures using MediaPipe and counts fingers.
        """
        mp_hands = mp.solutions.hands
        hands = mp_hands.Hands(static_image_mode = False,
                               max_num_hands = 1,
                               min_detection_confidence = 0.7, 
                               min_tracking_confidence = 0.5)
        mp_drawing = mp.solutions.drawing_utils

        last_fingers = -1
        exit = 0

        while not exit:
            with self.lock_frame:
                frame_L = self.frame_left.copy() if self.frame_left is not None else None
                frame_R = self.frame_right.copy() if self.frame_right is not None else None

            fingers_left = -1
            fingers_right = -1
            
            # Process left camera
            if frame_L is not None:
                image_rgb_L = cv2.cvtColor(frame_L, cv2.COLOR_BGR2RGB)
                results_L = hands.process(image_rgb_L)
                if results_L.multi_hand_landmarks:
                    for hand_landmarks in results_L.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame_L, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                        fingers_left = self.fingerCount(hand_landmarks)

            # Process right camera
            if frame_R is not None:
                image_rgb_R = cv2.cvtColor(frame_R, cv2.COLOR_BGR2RGB)
                results_R = hands.process(image_rgb_R)
                if results_R.multi_hand_landmarks:
                    for hand_landmarks in results_R.multi_hand_landmarks:
                        mp_drawing.draw_landmarks(frame_R, hand_landmarks, mp_hands.HAND_CONNECTIONS)
                        fingers_right = self.fingerCount(hand_landmarks)

            # Choose the result with more fingers detected (or any preferred)
            if fingers_left >= fingers_right and fingers_left != -1:
                self.fingers = fingers_left
                frame_to_show = frame_L
            elif fingers_right != -1:
                self.fingers = fingers_right
                frame_to_show = frame_R
            else:
                self.fingers = -1
                frame_to_show = frame_L if frame_L is not None else frame_R
            
            if frame_to_show is not None:
                cv2.putText(frame_to_show, f'Fingers: {self.fingers}', (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
                cv2.imshow('Hand Detection', frame_to_show)

            if self.fingers != last_fingers:
                last_fingers = self.fingers
                print(f"If you want {self.fingers} items, press ENTER. Otherwise, press ESC")

            key = cv2.waitKey(1) & 0xFF

            if key == 13:       # ENTER
                exit = 1
                
            elif key == 27:     # ESC
                self.fingers = -2
                exit = 1
    
        hands.close()
        
        self.gestureOn = False

        if frame_to_show is not None:
            cv2.destroyWindow('Hand Detection')

    def getCentroid(self, frame):
        """
        Detects the centroid of a green object in the frame.
        """
        hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)
                
        # Define the green color range in HSV
        lower_green = np.array([35, 50, 200])   # Minimum green value
        upper_green = np.array([80, 255, 255])  # Maximum green value
        
        # Create a mask to filter pixels within the green color range
        mask = cv2.inRange(hsv, lower_green, upper_green)
        
        # Filter noise using morphological operations
        kernel = np.ones((5, 5), np.uint8)
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, kernel)
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
        
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        best_bbox = None
        
        # Filter contours by size and aspect ratio to avoid noise
        for contour in contours:
            x, y, w, h = cv2.boundingRect(contour)
            area = w * h
            aspect_ratio = w / float(h)
            
            # Filter by size and aspect ratio to avoid incorrect detections
            if 300 < area < 8000 and 0.5 < aspect_ratio < 1.5:
                best_bbox = (x, y, w, h)
        
        if best_bbox is not None:
            p1 = (int(best_bbox[0]), int(best_bbox[1]))
            p2 = (int(best_bbox[0] + best_bbox[2]), int(best_bbox[1] + best_bbox[3]))
            cv2.rectangle(frame, p1, p2, (255, 0, 0), 2, 1)
            
            # Calculate centroid of the bounding box
            centroid_x = int((p1[0] + p2[0]) / 2)
            centroid_y = int((p1[1] + p2[1]) / 2)
            centroid = (centroid_x, centroid_y)

            return centroid 
        
        else:
            return None
            
    def triangulation(self, centroid_L, centroid_R):
        """
        Performs triangulation to estimate the 3D position of a point.
        """
        point_3d = None

        display_L = self.frame_left.copy()
        display_R = self.frame_right.copy()

        if centroid_L is not None:
            cv2.circle(display_L, centroid_L, 5, (0, 255, 0), -1)   # centroid_L[0]
        if centroid_R is not None:
            cv2.circle(display_R, centroid_R, 5, (0, 255, 0), -1)   # centroid_R[0]

        cv2.imshow("Left Camera Centroid", display_L)
        cv2.imshow("Right Camera Centroid", display_R)

        if centroid_L is not None and centroid_R is not None:
            # Undistort points
            ptsL_np = np.array(centroid_L, dtype=np.float32).reshape(-1, 1, 2)
            ptsR_np = np.array(centroid_R, dtype=np.float32).reshape(-1, 1, 2)
            undistL = cv2.undistortPoints(ptsL_np, self.K1, self.D1, P=self.K1)
            undistR = cv2.undistortPoints(ptsR_np, self.K2, self.D2, P=self.K2)

            # Projection matrix for each camera
            P1 = np.hstack((np.eye(3), np.zeros((3, 1))))
            P2 = np.hstack((self.R, self.T))
            P1 = self.K1 @ P1
            P2 = self.K2 @ P2

            # Triangulation
            point_4d = cv2.triangulatePoints(P1, P2, undistL, undistR)
            point_3d = point_4d[:3] / point_4d[3]

            print(f"Estimated 3D coordinates: {point_3d.ravel()}")
            self.triangulationOn = False

        return point_3d
        
    def centroidAdquisition(self):
        """
        Acquires centroids from both cameras and performs triangulation.
        """
        centroid_L = self.getCentroid(self.frame_left)
        centroid_R = self.getCentroid(self.frame_right)
        
        """
        if not centroid_L or not centroid_R:
            print("ERROR: Centroid not detected in one or both cameras.")
            c.put(-1)
        """
        
        while centroid_L is None and centroid_R is None:
            centroid_L = self.getCentroid(self.frame_left)
            centroid_R = self.getCentroid(self.frame_right)
            print("Couldn't detect object")
            time.sleep(1)

        print("Object detected, triangulation started")

        triangulatedPosition = self.triangulation(centroid_L, centroid_R)
        
        if triangulatedPosition is None:
            print("ERROR: Triangulation failed.")
            self.position = None
        
        self.position = triangulatedPosition # x, y, z
            
    def sendPosition(self, ip, topic, pos):
        """
        Sends the 3D position to RoboDK via MQTT.
        """
        client = mqtt.Client()
        try:
            client.connect(ip, 1883, 60)
            payload = json.dumps({"x": float(pos[0]), "y": float(pos[1]), "z": float(pos[2])})
            client.publish(topic, payload)
            print(f"Sent to RoboDK: {payload}")
            client.disconnect()
        except Exception as e:
            print(f"ERROR: Could not send message to RoboDK: {e}")
       
    # Callback when the client receives a CONNACK response from the server
    def on_connect(self, client, userdata, flags, rc):
        print(f"Connected with result code {rc}")
        client.subscribe(self.receiveTopic)   
        
    # Callback when a message is received on a subscribed topic
    def on_message(self, client, userdata, msg):
        payload = msg.payload.decode('utf-8')
        try:
            self.robotState = bool(int(payload)) 
            print(f"Received from RoboDK: {self.robotState}")
            self.received = True
        except ValueError:
            print("Received invalid payload. Expected 0 or 1.")
        
    def getRobotInfo(self, ip):
        """
        Retrieves the robot's state from RoboDK via MQTT.
        """
        client = mqtt.Client()
        client.on_connect = self.on_connect
        client.on_message = self.on_message

        try:
            client.connect(ip, 1883, 60)
            client.loop_start()

            # Wait until we receive the message
            while self.robotState:
                time.sleep(1)

            client.loop_stop()
            client.disconnect()

        except Exception as e:
            print(f"ERROR: Could not connect to RoboDK: {e}")
    
    def releaseAll(self):
        """
        Releases all resources and stops threads.
        """
        self.stopThreads = True
        self.cap_L.release()
        self.cap_R.release()
        cv2.destroyAllWindows()

    def videoCapture(self):
        while self.robotState:
            # Capture one image from each camera
            with self.lock_frame:
                img_L = self.frame_left
                img_R = self.frame_right

            if img_L is None or img_R is None:
                print("ERROR: No image captured.")
                continue

            # Resize images to same size
            resized_L = cv2.resize(img_L, (self.width, self.height))
            resized_R = cv2.resize(img_R, (self.width, self.height))

            # Combine both images side-by-side
            combined = np.hstack((resized_L , resized_R))
            cv2.imshow(self.window, combined)
            cv2.waitKey(1)
            
        cv2.destroyAllWindows()

    def robotCommunication(self):
        while self.robotState:
            if not self.gestureOn:
                if self.fingers == -1:
                    print("ERROR: Detection failed, exiting...")
                elif self.fingers == -2:
                    print("Exiting the program...")
                else:
                    centroidThread = threading.Thread(target = self.centroidAdquisition)
                    centroidThread.start()

                    centroidThread.join()

                    if self.position is not None:

                        self.sendPosition(self.ip, self.sendTopic, self.position)
                        time.sleep(1)

                        print(f"Robot picking up {self.fingers} at position [{self.position[0]}, {self.position[1]}, {self.position[2]}]")
                        self.getRobotInfo(self.ip)

                        print("Process completed, closing program...")

            time.sleep(1)

    def mainProgram(self):
        """
        Main program logic for gesture detection, centroid acquisition, and robot communication.
        """
        cv2.namedWindow(self.window, cv2.WINDOW_NORMAL)
        self.loadCalibration()
        
        camera_L = threading.Thread(target = self.capture_L)
        camera_R = threading.Thread(target = self.capture_R)
        camera_L.start()
        camera_R.start()

        time.sleep(1)        
        
        print("Indicate with your hand how many items you want to pick up. Press ESC to exit")        
        gestures = threading.Thread(target = self.gestureDetection)  
        gestures.start()
        
        robot_communication = threading.Thread(target = self.robotCommunication)  
        robot_communication.start()

        self.videoCapture()

        gestures.join()
        robot_communication.join()
        camera_L.join()
        camera_R.join()
        self.releaseAll()

In [ ]:
if __name__ == "__main__":  
    # Start message to indicate that necessary files are being verified
    print("\n\nVerifying calibration files...")

    # Create an instance of the calibrator with chessboard parameters
    calibration = cameraCalibration()

    # Variable to determine if any camera needs to be recalibrated
    recalibrate = False

    # === Verify and calibrate the left camera if necessary ===
    if not os.path.exists(CALIBRATION_PATH_L):
        recalibrate |= not calibration.individual_calibration("Left", CALIBRATION_IMG_PATH_L, CALIBRATION_PATH_L)

    # === Verify and calibrate the right camera if necessary ===
    if not os.path.exists(CALIBRATION_PATH_R):
        recalibrate |= not calibration.individual_calibration("Right", CALIBRATION_IMG_PATH_R, CALIBRATION_PATH_R)

    # If individual calibration files still do not exist after attempting to calibrate, exit with an error
    if not os.path.exists(CALIBRATION_PATH_L) or not os.path.exists(CALIBRATION_PATH_R):
        print("ERROR: Individual calibration failed.")
        exit()

    # === Verify and calibrate the stereo system if necessary ===
    # This is done if the stereo calibration file does not exist or if any camera was recalibrated
    if not os.path.exists(CALIBRATION_PATH_STEREO) or recalibrate:
        if not calibration.stereo_calibration(
            CALIBRATION_PATH_R, 
            CALIBRATION_PATH_L,
            STEREO_IMG_PATH_R, 
            STEREO_IMG_PATH_L,
            CALIBRATION_PATH_STEREO
        ):
            print("ERROR: Stereo calibration failed.")
            exit()
    
    # Create an instance of the stereo interface        
    mainProgram = stereoInterface()
    
    # Run the main program logic
    mainProgram.mainProgram()



Verifying calibration files...



Initializing video



